In [1]:
import chess.pgn
import numpy as np
from IPython.display import display, clear_output
import torch
from fastai2.data import *
from fastai2.basics import *
from fastai2.callback.all import *

In [10]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
def get_bitboard(board):
    '''
    params
    ------
    board : chess.pgn board object
        board to get state from
    returns
    -------
    bitboard representation of the state of the game
    64 * 6 + 5 dim binary numpy vector
    64 squares, 6 pieces, '1' indicates the piece is at a square
    5 extra dimensions for castling rights queenside/kingside and whose turn
    '''
    bitboard = np.zeros(64*6*2+5)

    piece_idx = {'p': 0, 'n': 1, 'b': 2, 'r': 3, 'q': 4, 'k': 5}

    for i in range(64):
        if board.piece_at(i):
            color = int(board.piece_at(i).color) + 1
            bitboard[(piece_idx[board.piece_at(i).symbol().lower()] + i * 6) * color] = 1

    bitboard[-1] = int(board.turn)
    bitboard[-2] = int(board.has_kingside_castling_rights(True))
    bitboard[-3] = int(board.has_kingside_castling_rights(False))
    bitboard[-4] = int(board.has_queenside_castling_rights(True))
    bitboard[-5] = int(board.has_queenside_castling_rights(False))

    return bitboard

In [3]:
def get_result(game):
    result = game.headers['Result']
    result = result.split('-')
    if result[0] == '1':
        return 1
    elif result[0] == '0':
        return -1
    else:
        return 0

In [41]:
# Change function to write files with correct names 
def pgn_to_np(path):
    games = open(path)
    bitboards = []
    labels = []
    num_games = 0

#     while True:
    for i in range(4):
        if num_games % 1 == 0:
            clear_output()
#             print(num_games)
        num_games += 1

        game = chess.pgn.read_game(games)

        try:
            result = get_result(game)
            board = game.board()
            for move in game.mainline_moves():
                board.push(move)
                bitboard = get_bitboard(board)

                bitboards.append(bitboard)
                labels.append(result)
        except:
            clear_output()
            print(f"{num_games} games in file")
            break

#     bitboards = np.array(bitboards)
#     labels = np.array(labels)

#     np.save('data/bitboards.npy', bitboards)
#     np.save('data/labels.npy', labels)

    return bitboards, labels

bitboards, labels = pgn_to_np('data/games.pgn')

# bitboards = np.load('data/bitboards.npy')
# labels = np.load('data/labels.npy')

In [22]:
class ChessDataset(torch.utils.data.Dataset):
    def __init__(self, bitboards, labels):
        self.x = bitboards
        self.y = labels
        self.c = 3

    def __getitem__(self, index):
        return torch.from_numpy(self.x[index]).type(torch.FloatTensor).to(device)

    def __len__(self):
        return self.x.shape[0]

In [23]:
data = ChessDataset(bitboards[:100], labels[:100])

In [24]:
dl = torch.utils.data.DataLoader(data, batch_size=8, shuffle=True)

In [26]:
dls = DataLoaders(dl, dl)

In [27]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear1 = nn.Linear(773, 128)
        self.linear2 = nn.Linear(128, 1)

    def forward(self):
        hidden1 = self.linear1(feature_vec).clamp(min=0)  # ReLU
        output = self.linear2(hidden1)
        return F.log_softmax(output, dim=1)

In [28]:
learn = Learner(dls, SimpleModel(), loss_func=F.nll_loss, opt_func=Adam, metrics=accuracy)

In [29]:
learn.fit(1)

TypeError: forward() takes 1 positional argument but 8 were given